## Basic Algorithms: Fibonacci

Another form of recursion is tree recursion. Consider computing a fibonacci sequence, in which each number is the sum of the previous two, with the first two taken to be 0 and 1.

### Recursive Fibonacci. 


We write Fibonacci recursively with the first two numbers as base cases.

![](https://mitpress.mit.edu/sicp/full-text/book/ch1-Z-G-13.gif)

(from SICP)

Signature: `def fib_recursive(n)`

In [9]:
def fib_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fib_recursive(n-1) + fib_recursive(n-2)

for i in [1, 2, 7, 13, 29, 33]:
    print(fib_recursive(i))

0
1
8
144
317811
2178309


### Q1. 

What are the space and time complexities of this implementation? Hint: think aboutthe number of items in a binary tree and its depth.

<font color = "darkblue">
***your answer here***

1) time complexity:
Take advantage of recursion tree. 
T(n) = T(n-1) + T(n-2) + C (n>=2)
From the root of the tree, every node generates no more than 2 nodes, and usually 2 nodes(n>=2).
Going down from the root to leaves, the biggest n on each layer reduces 1, so the length of the tree is n. every node takes time C(constant), and the total number of nodes (time) are no more than 1 + 2 + 2 ^ 2 + ... + 2^(n - 1) = 2^n - 1 = O(2^n).
so the upper bound of time complexity is O(2^n).

2) space complexity:
The depth of the recursion tree is n. Look at the deepest path(the most left one). It has n-1 nodes(stack frames), each of which costs space C, so space complexity is C*(n-1) = O(n).


### Dynamic Programming and Iteration

From Skiena
>..**dynamic programming**, which typically removes one element from the problem, solves the smaller problem, and then uses the solution to this smaller problem to add back the element in the proper way. **Divide-and-conquer** instead splits the problem in (say) halves, solves each half, then stitches the pieces back together to form a full solution.

>Dynamic programming is a technique for efficiently implementing a recursive algorithm by storing partial results. The trick is seeing whether the naive recursive algorithm computes the same subproblems over and over and over again. If so, storing the answer for each subproblems in a table to look up instead of recompute can lead to an efficient algorithm. Start with a recursive algorithm or definition. Only once we have a correct recursive algorithm do we worry about speeding it up by using a results matrix. Dynamic programming is generally the right method for optimization problems on combinatorial objects that have an inherent left to right order among components. Left-to-right objects includes: character strings, rooted trees, polygons, and integer sequences.

### Q2.

Here is an implementation of Fibonacci using dynamic programming: they key is to notice that the recurrence we used can be put into an iterative form and just stored in an ever increasing array. What is the space and time complexity here? What if you were somehow able to save the array outside of the function when u calculate `fib(M)` and subsequently had to calculate `fib(N)`, where $N>M$.

In [7]:
def fib_iterative(n):
    fibs=[]
    fibs.append(0)
    fibs.append(1)
    for i in range(2, n):
         fibs.append(fibs[i-1]+fibs[i-2])
    return fibs[n-1]

In [8]:
for i in [1, 2, 7,13,29,33]:
    print(fib_iterative(i))

0
1
8
144
317811
2178309


<font color = "darkblue">
***your answer here***

1) time complexity
The for-loop within the function loops n-1 times, and constant time C(sum and append operations) for each. So the time complexity is C*(n-1) + C = O(n).

2) space complexity
The biggest space is the fibs[], and it has n elements in the end. So the space complexity is O(n).

### Recursive Fibonacci with cacheing

### Q3. 

Use `cache` and `call_counter` as decorators on `fib_recursive` and print the fibonacci numbers for 7,13,29, 33. What order should these decorators be called to make sure `call_counter` gets the actual number of calls to `fib_recursive`?

We've written the `cache` decorator for you. You have to write the `call_counter` decorator which takes the function as argument and using a `count_dictionary` whose keys are function names, counts the number of times the function is called.

In [3]:
def cache(f):
    """a single argument function whose values may be cached"""
    cache = {}
    def memoized_func(x):
        if x not in cache:
            cache[x] = f(x)
        return cache[x]
    memoized_func.__name__ = f.__name__
    return memoized_func
    

In [4]:
def call_counter(count_dictionary):
    def call_counter_decorator(func):
        def inner(*args, **kwargs):
            name = func.__name__
            ######your code here
            #print("name:" + name)
            if name not in count_dictionary:
                count_dictionary[name] = 0
            count_dictionary[name] = count_dictionary[name] + 1
            #print(args)
            #print(count_dictionary)
            return func(*args, **kwargs)
        return inner
    return call_counter_decorator


In [7]:
######your code here
ccounter={}
@cache
@call_counter(ccounter)    

def fib_recursive(n):      #terminal condition in the following original example is inconsistent
    if n == 0:
        return 0
    if n == 1:
        return 1
    return fib_recursive(n-1) + fib_recursive(n-2)

for i in [7,13,29, 33]:
#for i in [7]:
    print(i, fib_recursive(i), ccounter['fib_recursive'])

(7, 13, 8)
(13, 233, 14)
(29, 514229, 30)
(33, 3524578, 34)


<font color = "darkblue">
***your answer here***

The call_counter() should be closer to the fib fuction if you want to counts the calls of 
the actual recursived fib(). Although it still works if cache is closer to the fib fuction, 
the call_counter() returns the counts of memorized fib() instead of the actual recursived fib(). The terminal condition in the following original example code is inconsistent to the function call tree, so I write a new one.

In [8]:
ccounter={}             # the original codes in the homework
@cache
@call_counter(ccounter)
def fib_recursive(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    return fib_recursive(n-1) + fib_recursive(n-2)

for i in [7,13,29, 33]:
#for i in [7]:
    print(i, fib_recursive(i), ccounter['fib_recursive'])
    

(7, 8, 7)
(13, 144, 13)
(29, 317811, 29)
(33, 2178309, 33)


### Q4.

What is the time and space complexity of the the memoized Fibonacci? HINT: assume evaluation happens left to right on sub-expressions, so that `fib_recursive(n-1)` side of the tree is evaluated first, and thus the tree is evaluated depth first, from left to right. What kind of pruning happens in the tree?

<font color = "darkblue">
***your answer here***

Look at the recursion tree. 

Every layer: For each node fib_iter(n), the left child is fib_iter(n - 1), and the right is fib_iter(n - 2). The recursion call progress goes along the left down to the leaf the then go to the right(like the DFS). So every time it comes to the right node, it will find the result has already been in the cache(), so it returned its value and won't go deeper. That means every layer actually has 2 nodes of cache(fib()). 

Depth: The leftest(the longest) path has n nodes, so the depth is n.


1)time complexity: 

if you counts the call numbers of cache(fib()), it
equals to the total amounts of nodes in the pruned tree. 2 * (n - 1) + C  = O(n);

if you counts the call numbers of inner actual recursive fib(), it
equals to the nodes in the leftest path. n = O(n).

2)space complexity:
Equals to the cache{} + C = O(n)



### Q5.

Do you really need to store the entire array in the dynamic programming implementation? Isnt it enough to have only saved the previous two Fibonacci numbers? Implement such an algorithm in `fib_iterative2(n)`. What is its space and time complexity? How is the time complexity different from that of the previous iterative attempt?


In [4]:
#your code here
def fib_iterative2(n):
    if n == 0: 
        return 0
    if n == 1:
        return 1
    prev = (0, 1)
    for i in range(2, n + 1):
        new = prev[0] + prev[1]
        prev = (prev[1], new)
    return prev[1]

In [10]:
#your code here
for i in [1, 2, 7,13,29,33]:
    print(fib_iterative(i))

0
1
8
144
317811
2178309


<font color = "darkblue">
***your answer here***

space complexity: constant, O(1)

time complexity: O(n), time complexity of the two implementations are the same order of n

### Q6.

Write an algorithm for insertion sort.

![](https://camo.githubusercontent.com/8f6fedc10da579f13b22b949f6ad29255b6d721f/68747470733a2f2f75706c6f61642e77696b696d656469612e6f72672f77696b6970656469612f636f6d6d6f6e732f302f30662f496e73657274696f6e2d736f72742d6578616d706c652d33303070782e676966)

(from wikipedia)

The algorithm is also illustrated here: http://cs.armstrong.edu/liang/animation/web/InsertionSort.html and may be described thus:

Insertion sort is a method for sorting that starts with a single element (thus forming a trivially sorted list) and then incrementally inserts the remaining elements so that the list stays sorted.

Talk about the best, worst and average complexity of insertion sort. Use the A=[5,2,1,3,8,6,9] to show your code!

In [1]:
#your code here
def insert_sort(list_to_sort):
    for i in range(1, len(list_to_sort)):
        j = i
        while (j > 0 and list_to_sort[j] < list_to_sort[j - 1]):
            (list_to_sort[j], list_to_sort[j - 1]) = (list_to_sort[j - 1], list_to_sort[j])
            j -= 1
    return list_to_sort

A = [5,2,1,3,8,6,9]
insert_sort(A)
    


[1, 2, 3, 5, 6, 8, 9]

<font color = "darkblue">
***your answer here***

1) time complexity: 
the outer loop has i = (n - 1) loops and the inner one has (i - 1) loops for the worst case, 
so (n - 2) + (n - 3) + ... + 1 = (n - 1) * (n - 2) / 2 = O(n^2).

2) space complexity:
if not consider the space that the input array costs, there is no extra space. 
So space complexity is O(1).
